In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `bootpython && heroku run --app guarded-everglades-89687 make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run --app guarded-everglades-89687 make importdb && make pipeline`
# (6) Remember to delete contents of ~/Documents/links.txt
links = pd.read_csv('data/export.csv')
links

/var/folders/7f/6ty6wfcd1hs3d1wz9b1c4gsw0000gn/T/ipykernel_17025/1024356683.py:12: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108736,173411,https://globalshield.substack.com/p/gcr-policy...,GCR Policy Newsletter (26 June 2023),\N,\N,2024-01-24 19:58:34.060946+00,2024-01-24 19:58:34.060946+00,NaN,\N,GlobalShield,41.0,0.0,\N
108737,173412,https://globalshield.substack.com/p/gcr-policy...,GCR Policy Newsletter (10 July 2023),\N,globalshield.substack.com,2024-01-24 19:58:34.06203+00,2024-01-24 20:20:13.311344+00,-1.0,\N,GlobalShield,88.0,0.0,\N
108738,173413,https://globalshield.substack.com/p/gcr-policy...,GCR Policy Newsletter (13 June 2023),\N,globalshield.substack.com,2024-01-24 19:58:34.06315+00,2024-01-24 20:20:26.695123+00,-1.0,\N,GlobalShield,67.0,0.0,\N
108739,173414,https://cset.georgetown.edu/newsletter/novembe...,A pivotal moment for AI regulation — Biden pus...,\N,cset.georgetown.edu,2024-01-24 19:58:34.064232+00,2024-01-24 20:20:00.778291+00,-1.0,\N,CSET,98.0,0.0,\N


In [2]:
links['added'].max()

'2024-01-24 19:58:34.065345+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299        1306
1300        1307
1301        1308
1302        1309
1303        1310
           ...  
108685    173359
108686    173360
108687    173361
108688    173363
108689    173364
Name: id, Length: 15852, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred


In [5]:
links[links['url'].duplicated()]['id']

Series([], Name: id, dtype: int64)

In [6]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'starred': '\\N',
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = pd.concat([links, pd.DataFrame([entry])], ignore_index=True)

links

...1/30
...11/30
...21/30
...30/30


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108766,173441,https://experiencemachines.substack.com/p/ai-s...,AI systems as real-life thought experiments ab...,\N,\N,2024-01-26 13:16:26.276228,2024-01-26 13:16:26.276235,\N,\N,Custom,36.0,0.0,\N
108767,173442,https://arxiv.org/pdf/2311.08576.pdf,2311.08576.pdf,\N,\N,2024-01-26 13:16:26.284441,2024-01-26 13:16:26.284449,\N,\N,Custom,41.0,0.0,\N
108768,173443,https://www.ncsc.gov.uk/report/impact-of-ai-on...,The near-term impact of AI on the cyber threat...,\N,\N,2024-01-26 13:16:26.291820,2024-01-26 13:16:26.291827,\N,\N,Custom,35.0,0.0,\N
108769,173444,https://osf.io/preprints/osf/ng7pu,OSF Preprints,\N,\N,2024-01-26 13:16:26.300463,2024-01-26 13:16:26.300471,\N,\N,Custom,43.0,0.0,\N


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108766,173441,https://experiencemachines.substack.com/p/ai-s...,AI systems as real-life thought experiments ab...,\N,\N,2024-01-26 13:16:26.276228,2024-01-26 13:16:26.276235,\N,\N,Custom,36.0,0.0,\N
108767,173442,https://arxiv.org/pdf/2311.08576.pdf,2311.08576.pdf,\N,\N,2024-01-26 13:16:26.284441,2024-01-26 13:16:26.284449,\N,\N,Custom,41.0,0.0,\N
108768,173443,https://www.ncsc.gov.uk/report/impact-of-ai-on...,The near-term impact of AI on the cyber threat...,\N,\N,2024-01-26 13:16:26.291820,2024-01-26 13:16:26.291827,\N,\N,Custom,35.0,0.0,\N
108769,173444,https://osf.io/preprints/osf/ng7pu,OSF Preprints,\N,\N,2024-01-26 13:16:26.300463,2024-01-26 13:16:26.300471,\N,\N,Custom,43.0,0.0,\N


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108766,173441,https://experiencemachines.substack.com/p/ai-s...,AI systems as real-life thought experiments ab...,\N,\N,2024-01-26 13:16:26.276228,2024-01-26 13:16:26.276235,\N,\N,Custom,36,0,\N
108767,173442,https://arxiv.org/pdf/2311.08576.pdf,2311.08576.pdf,\N,\N,2024-01-26 13:16:26.284441,2024-01-26 13:16:26.284449,\N,\N,Custom,41,0,\N
108768,173443,https://www.ncsc.gov.uk/report/impact-of-ai-on...,The near-term impact of AI on the cyber threat...,\N,\N,2024-01-26 13:16:26.291820,2024-01-26 13:16:26.291827,\N,\N,Custom,35,0,\N
108769,173444,https://osf.io/preprints/osf/ng7pu,OSF Preprints,\N,\N,2024-01-26 13:16:26.300463,2024-01-26 13:16:26.300471,\N,\N,Custom,43,0,\N


In [10]:
links.to_csv('data/export.csv', index=False)